In [1]:

from owlready2 import Thing, get_ontology, AllDisjoint, sync_reasoner_pellet, Imp, FunctionalProperty
from owlready2 import *

# Create a new ontology
onto = get_ontology("http://example.org/onto#")

# Define classes for the ontology based on the CSV file contents
with onto:
    class TVShow(Thing): pass
    class Episode(Thing): pass
    class User(Thing): pass
    class Comment(Thing): pass
    class Rating(Thing): pass
    class Emotion(Thing): pass

    # Define properties
    class aPourTitre(TVShow >> str, FunctionalProperty): pass

    class aPourTVShow(Episode >> TVShow, FunctionalProperty): pass
    class aPourSaison(Episode >> int, FunctionalProperty): pass
    class aPourNumero(Episode >> int, FunctionalProperty): pass
    class rewatchedCount(Episode >> int, FunctionalProperty): pass
    # class aPourDate(Episode >> str, FunctionalProperty): pass
    class aPourAnnee(Episode >> int, FunctionalProperty): pass
    class aPourMois(Episode >> int, FunctionalProperty): pass

    class aPourNote(Rating >> int, FunctionalProperty): pass
    # class aPourDate(Rating >> str, FunctionalProperty): pass

    class aPourContenu(Comment >> str, FunctionalProperty): pass
    class nbLikes(Comment >> int, FunctionalProperty): pass
    # class aPourDate(Comment >> str, FunctionalProperty): pass
    
    class aPourEmotion(Emotion >> str, FunctionalProperty): pass
    # class aPourDate(Emotion >> str, FunctionalProperty): pass
    
    # Define relationships
    class is_episode_of(Episode >> TVShow): pass
    
    class posts_by(Comment >> User): pass
    class user_posts_comment(User >> Comment):
        inverse = posts_by
    class comments_on(Comment >> Episode): pass
    class is_commented_on_by(Episode >> Comment):
        inverse = comments_on

    
    class follows(User >> TVShow): pass
    class has_started(User >> TVShow): pass
    class has_stopped(User >> TVShow): pass
    class is_stopped_by(TVShow >> User): 
        inverse = has_stopped
    class has_watched(User >> Episode): pass

    class rated_by_user(Rating >> User): pass
    class user_rates(User >> Rating):
        inverse = rated_by_user
    class rates_episodes(Rating >> Episode): pass
    class is_rated_by(Episode >> Rating):
        inverse = rates_episodes

    class expressed_by(Emotion >> User): pass
    class user_expresses(User >> Emotion):
        inverse = expressed_by
    class evokes(Emotion >> Episode):  pass
    class is_evoked_by(Episode >> Emotion):
        inverse = evokes

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
import csv

def load_data(filename, process_row):
    with open(filename, 'rt', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=',')
        count = 0
        for row in reader:
            if count > 0 and count%100000 == 0:
                print(f'Processed {count} rows')
            process_row(row)
            count += 1

In [3]:
import datetime

def process_tv_shows_episodes(row):
    tv_show = onto.TVShow(name=row['tv_show_name'])
    tv_show.aPourTitre = row['tv_show_name']

    episode = onto.Episode(name=row['episode_id'])
    episode.aPourTVShow = tv_show
    episode.is_episode_of.append(tv_show)
    episode.aPourSaison = int(row['episode_season_number'])
    episode.aPourNumero = int(row['episode_number'])
    episode.rewatchedCount = int(row['rewatched_count'])
    # episode.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()) # cast datetime to string to avoid error
    date = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S')
    annee = date.year
    episode.aPourAnnee = int(annee)
    mois = date.month
    episode.aPourMois = int(mois)

    user = onto.User(name=row['user_id'])
    user.has_started.append(tv_show) # user has started watching the tv show because they have seen at least one episode
    user.has_watched.append(episode) # user has watched the episode

path = './data/seen_episode_modified.csv'
load_data(path, process_tv_shows_episodes)
print(f'Loaded {len(onto.TVShow.instances())} TV shows and {len(onto.Episode.instances())} episodes seen and {len(onto.User.instances())} users')

Loaded 106 TV shows and 5227 episodes seen and 1 users


In [4]:
def process_episodes_ratings(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    rating = onto.Rating(name=row['vote_key'])
    rating.aPourNote = int(row['note'])
    # rating.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()) # cast datetime to string to avoid error

    rating.rated_by_user.append(user)
    rating.rates_episodes.append(episode)

path = './data/ratings_episode_votes_modified.csv'
load_data(path, process_episodes_ratings)
print(f'Loaded {len(onto.Rating.instances())} ratings')

Loaded 339 ratings


In [5]:
def process_tv_shows_followings(row):
    tv_show = onto.TVShow(name=row['tv_show_name'])
    tv_show.aPourTitre = row['tv_show_name']

    user = onto.User(name=row['user_id'])
    user.follows.append(tv_show)
    if row['active'] == '0' or row['archived'] == '1': # user has stopped following the tv show
        user.has_stopped.append(tv_show)

path = './data/followed_tv_show_modified.csv'
load_data(path, process_tv_shows_followings)
print(f'Loaded {len(onto.User.instances())} users and {len(onto.TVShow.instances())} TV shows')

Loaded 1 users and 119 TV shows


In [6]:
def process_episodes_comments(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    comment = onto.Comment(name=row['comment_id'])
    comment.aPourContenu = row['comment']
    comment.nbLikes = int(row['nb_likes'])
    # comment.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()) # cast datetime to string to avoid error

    comment.posts_by.append(user)
    comment.comments_on.append(episode)

path = './data/episode_comment_modified.csv'
load_data(path, process_episodes_comments)
print(f'Loaded {len(onto.Comment.instances())} comments')

Loaded 35 comments


In [7]:
def process_episodes_emotions(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    emotion = onto.Emotion(name=row['emotion_vote_id'])
    emotion.aPourEmotion = row['emotion_id']
    # emotion.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date())
    emotion.expressed_by.append(user)
    emotion.evokes.append(episode)

path = './data/emotions_episode_votes_modified.csv'
load_data(path, process_episodes_emotions)
print(f'Loaded {len(onto.Emotion.instances())} emotions')

Loaded 341 emotions


In [8]:
sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

* Owlready * Adding relation onto.34759499 aPourContenu Damn, Stephan breaking the circle of violence was breathtaking\°46 respect bruv
* Owlready * Adding relation onto.32426747 aPourContenu So sad we don\°31t see Kim and Nicholas sitting together in the bar and talking as friends \?75\?42 I would have shed a tear
* Owlready * Adding relation onto.33337268 aPourContenu I\°31m starting to get bored but what a plot it would be to see zacarias destroying the trafiquants because they try to black mail him
* Owlready * Adding relation onto.32406536 aPourContenu Damn who is spying who it\°31s more complicated that I thought
* Owlready * Adding relation onto.32606931 aPourContenu I love so much the mentality : they are all awesome but there also all fans of each other and they respect everyone some much I love this sport\°31s mentality !
* Owlready * Adding relation onto.33012136 aPourContenu This guy is like the embodiment of wisdom \?75\?55
* Owlready * Adding relation onto.31913600 aPourC

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [9]:
onto.save(file = "tvtime_ontology.owl", format = "rdfxml")

# Créer des classes plus complexes

In [10]:
# Load the ontology
onto = get_ontology("tvtime_ontology.owl").load()

In [11]:
with onto:
    class User(Thing):
        equivalent_to = [User & (has_started.some(TVShow) & has_watched.some(Episode) & follows.some(TVShow) & has_stopped.some(TVShow)) & user_rates.some(Rating) & user_posts_comment.some(Comment) & user_expresses.some(Emotion)]

    class TVShow(Thing):
        equivalent_to = [TVShow & (follows.some(User) & has_started.some(User) & is_stopped_by.some(User))]

    class Episode(Thing):
        equivalent_to = [Episode & (has_watched.some(User) & is_rated_by.some(Rating) & is_commented_on_by.some(Comment) & is_evoked_by.some(Emotion))]

    class Comment(Thing):
        equivalent_to = [Comment & (posts_by.some(User) & comments_on.some(Episode))]

    class Rating(Thing):
        equivalent_to = [Rating & (rated_by_user.some(User) & rates_episodes.some(Episode))]

    class Emotion(Thing):
        equivalent_to = [Emotion & (expressed_by.some(User) & evokes.some(Episode))]

sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

In [24]:
with onto:

    class TVShowEngageUser(Thing): pass
        # A tv show engages a user if the user has started watching the tv show,
        # has watched at least one episode,
        # has posted a comment on an episode of the tv show and
        # follows the tv show

    #  # Define a function to check if a user engages with a TV show
    # def check_engagement(user, tv_show):
    #     for episode in user.has_watched:
    #         if episode.is_episode_of == tv_show and user in episode.comments_by:
    #             return True
    #     return False

    # # Iterate over each user and TV show
    # for user in onto.User.instances():
    #     for tv_show in onto.TVShow.instances():
    #         # Check if the user has started watching the TV show
    #         if user.has_started and user.has_started[0] == tv_show:
    #             # Check if the user has watched at least one episode and posted a comment on an episode of the TV show
    #             if check_engagement(user, tv_show):
    #                 # If all conditions are met, create an instance of TVShowEngageUser
    #                 TVShowEngageUser(tv_show)

    rule1 = Imp()
    rule1.set_as_rule('''
        has_started(?user, ?tv_show),
        has_watched(?user, ?episode),
        follows(?user, ?tv_show),
        user_posts_comment(?user, ?comment),
        comments_on(?comment, ?episode),
        is_episode_of(?episode, ?tv_show)
        -> TVShowEngageUser(?tv_show)
    ''')

sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

print(f'Number of TV shows that engage the user: {len(onto.TVShowEngageUser.instances())} : ')
for tv_show in onto.TVShowEngageUser.instances():
    print(f'{tv_show.aPourTitre} engages the user')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

Number of TV shows that engage the user: 16 : 
Westworld engages the user
Ozark engages the user
Barry engages the user
Cobra Kai engages the user
Top Boy engages the user
Monster (2022) engages the user
Bloodhounds (2023) engages the user
Black Knight engages the user
The Night Agent engages the user
A Spy Among Friends engages the user
Citadel engages the user
The Terminal List engages the user
The Marked Heart engages the user
Florida Man engages the user
Physical: 100 engages the user
Arnold engages the user


* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [15]:
with onto:
    class is_influenced_by(TVShow >> User): pass
        # This is the list of the TVShow associated to a user
        # in which the sum of the likes on the comments made by the user on the episodes of the TVShow is greater than 50

    for user in onto.User.instances():
        for tv_show in onto.TVShow.instances():
            sum_likes = 0
            for episode in user.has_watched:
                if tv_show in episode.is_episode_of and episode.is_commented_on_by :
                    # print(f'{episode.is_commented_on_by}')
                    # print(f'User {user} has commented on episode {episode} of TV show {tv_show}')
                    for comment in episode.is_commented_on_by:
                        # print(f'{comment.nbLikes}')
                        sum_likes += comment.nbLikes
            if sum_likes > 50:
                tv_show.is_influenced_by.append(user)

sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

print(f'Number of TV shows that influence the user: ')
for tv_show in onto.TVShow.instances() :
    if tv_show.is_influenced_by :
        print(f'{tv_show.aPourTitre} influenced by Romain')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

Number of TV shows that influence the user: 
The Night Agent influences by Romain
A Spy Among Friends influences by Romain


* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


# Ne fonctionne pas

In [ ]:
with onto:

    class EmotionalEpisode(Thing): pass
        # An episode is emotional if it has been rated by the user,  has been evoked by an emotion and has been commented on by the user

    rule2 = Imp()
    rule2.set_as_rule('''
        user_rates(?user, ?rating),
        rates_episodes(?rating, ?episode),
        user_expresses(?user, ?emotion),
        evokes(?emotion, ?episode),
        user_posts_comment(?user, ?comment),
        comments_on(?comment, ?episode)
        -> EmotionalEpisode(?episode)
    ''')

sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

print(f'Number of emotional episodes: {len(onto.EmotionalEpisode.instances())} : ')
for episode in onto.EmotionalEpisode.instances():
    print(f'Episode {episode.name} is emotional')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

OwlReadyJavaError: Java error message is:
Exception in thread "main" java.lang.OutOfMemoryError: GC overhead limit exceeded
	at java.util.ArrayList.<init>(Unknown Source)
	at com.clarkparsia.pellet.rules.rete.BetaNode.join(BetaNode.java:116)
	at com.clarkparsia.pellet.rules.rete.Interpreter.processBetaNodes(Interpreter.java:109)
	at com.clarkparsia.pellet.rules.rete.Interpreter.run(Interpreter.java:236)
	at com.clarkparsia.pellet.rules.ContinuousRulesStrategy.applyRete(ContinuousRulesStrategy.java:179)
	at com.clarkparsia.pellet.rules.ContinuousRulesStrategy.complete(ContinuousRulesStrategy.java:291)
	at org.mindswap.pellet.ABox.isConsistent(ABox.java:1423)
	at org.mindswap.pellet.ABox.isConsistent(ABox.java:1260)
	at org.mindswap.pellet.KnowledgeBase.consistency(KnowledgeBase.java:1987)
	at org.mindswap.pellet.KnowledgeBase.isConsistent(KnowledgeBase.java:2061)
	at pellet.PelletRealize.run(PelletRealize.java:70)
	at pellet.Pellet.run(Pellet.java:105)
	at pellet.Pellet.mai